In [3]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [1]:
# logistic regression
# 用来解决二分类问题


# 0) prepare data
bc = datasets.load_breast_cancer()          # binary classification problem
X, y = bc.data, bc.target

n_samples, n_features = X.shape
print(n_samples, n_features)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
# print(y_train)
# print(X_train)

# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)             # make feature with zero mean and unit variance
X_test = sc.transform(X_test)                   

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)             # transfer to a tensor with only one column, each value in one row
y_test = y_test.view(y_test.shape[0], 1)
print(y_train.shape)

# 1) model
# f = w * x + b, sigmoid at the end
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()              # so that LogisticRegression use the initialization of nn.Module
        self.Linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.Linear(x))
        return y_pred
    
# 2) loss and optimizer
learning_rate = 0.01
model = LogisticRegression(n_features)
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 3) training loop
epochs = 200
for epoch in range(epochs):
    # forward and loss
    y_pred = model.forward(X_train)
    loss = criterion(y_pred, y_train)
    
    # backward
    loss.backward()
    
    # update
    optimizer.step()
    optimizer.zero_grad()
    
    if epoch % 10 == 0:
        print(f'Epoch: {epoch // 10}, Loss: {loss:.4f}')

with torch.no_grad():
    y_pred = model.forward(X_test)         
    # print(y_pred)
    y_pred_cls = y_pred.round()
    # print(y_pred_cls)
    acc = y_pred_cls.eq(y_test).sum() / float(y_test.shape[0])
    
    print(f'Accuracy: {acc:.4f}')


NameError: name 'datasets' is not defined